# Intro

运用EMD-LSTM对股价波动率进行预测，进而对VaR值进行估计预测

在波动率预测方面，使用传统GARCH模型，单一LSTM模型与EMD-LSTM进行比对；

在VaR度量方面，使用GARCH模型对结果进行比较，Kupiec检验进行验证。

# Preperations

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from scipy.stats import binom, chi2
from arch import arch_model
from arch.unitroot import ADF
from arch.unitroot import engle_granger
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.arima import ARIMA
from arch.univariate import ARCH, GARCH

from pylab import mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator
import warnings
warnings.simplefilter(action='ignore', category= FutureWarning)
mpl.rcParams['font.sans-serif']=["SimSun"]

In [ ]:
from PyEMD import EMD

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import mean_squared_error, mean_absolute_error
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

from sklearn.model_selection import train_test_split

## Import Data

In [ ]:
path_file='...'

In [ ]:
li = pd.read_csv('LI.csv', usecols=lambda col: col != 'Datetime')

## Log returns and Concatenate

In [ ]:
mean = li['Close'].pct_change().mean()
li['return'] = li['Close'].pct_change().apply(lambda x: mean if pd.isna(x) else round(np.log(1 + x), 6))

In [ ]:
li.head()

## Realized Volatilities

In [ ]:
window_size = 5
li = li.astype(float)
# 计算历史波动率
li['rv'] = li.iloc[:, 2].rolling(window=window_size).std()
li['rv'] = li['rv']

In [ ]:
li['rv'].fillna(li['rv'].mean(), inplace=True)

In [ ]:
li.head()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(li['return'], label='Log Return')
plt.xlabel('Date')
plt.ylabel('Return')
plt.title('Log Return over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot( li['rv'], label='Realized Volatility')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.title('Realized Volatility over Time')
plt.legend()
plt.show()

# EMD-LSTM

## EMD

In [ ]:
emd = EMD(n_imfs=7)
rv=np.array(li['rv'].tolist())
imfs = emd(rv)
residue = emd.residue

In [ ]:
fig, axs = plt.subplots(nrows=8, ncols=1, figsize=(15, 10), sharex=True)

# 绘制 IMFs
for i, imf in enumerate(imfs[:7]):  # 限制绘制的 IMFs 数量为 7
    axs[i].plot(imf, label=f'IMF {i+1}')
    axs[i].set_ylabel('')  # 设置纵轴标签为空字符串
    axs[i].legend(fontsize='large')

# 绘制残差
axs[7].plot(residue, label='残差')
axs[7].set_ylabel('')
axs[7].legend()
axs[7].legend(fontsize='large')
# 设置 x 轴标签
axs[7].set_xlabel('日期序列')

plt.tight_layout()
plt.show()

In [ ]:
imfs = imfs[:7]

In [ ]:
imfs = pd.DataFrame(imfs.T, columns=[f'IMF_{i+1}' for i in range(7)])
residue = pd.DataFrame(residue, columns=['Residue'])

In [ ]:
df = pd.concat([imfs, residue], axis=1)
df.head()

## LSTM

In [ ]:
def get_imf(train_imfs, index):
    """
    获取指定 IMF 数据列

    train_imfs: DataFrame，包含 IMF 数据的 DataFrame
    index: int，IMF 编号
    
    返回：
    IMF 数据列
    """
    imf_column_name = f'IMF_{index}'
    return train_imfs[imf_column_name]

### Spliting dataset

In [ ]:
train_imfs, test_imfs = train_test_split(imfs, test_size=0.2, shuffle=False)
train_residue, test_residue = train_test_split(residue, test_size=0.2, shuffle=False)

In [ ]:
train_imfs.shape

In [ ]:
test_imfs.shape

In [ ]:
imf1 = get_imf(train_imfs, 1)
plt.figure(figsize=(15, 5))

plt.plot(np.arange(len(imf1)), imf1, color='blue', label='训练目标')

# 绘制测试目标
plt.plot(np.arange(len(train_imfs), len(train_imfs) + len(test_imfs)),
         test_imfs['IMF_1'], color='black', label='测试目标')

plt.title('IMF_1 数据划分情况', fontsize=20)
plt.xlabel('时间', fontsize=15)
plt.legend(loc='best')

# 设置 x 轴刻度间隔
locator = MultipleLocator(1000)
plt.gca().xaxis.set_major_locator(locator)
plt.show()

### Model training

In [ ]:
model = Sequential()

# 添加LSTM层
model.add(LSTM(units=4, input_shape=(10, 1)))

# 添加一层线性网络得到最终输出
model.add(Dense(units=1, kernel_regularizer=l2(0.01)))  # 加入L2正则化

# 编译模型
optimizer = Adam()
model.compile(optimizer=optimizer, loss='mse')  # 使用均方误差作为损失函数

# 输出模型结构
model.summary()

#### IMF1

In [ ]:
imf1 = get_imf(train_imfs, 1)
imf1_test= get_imf(test_imfs, 1)

In [ ]:
imf1_his=model.fit(imf1, np.roll(imf1, -1), epochs=100, batch_size=128)

In [ ]:
imf1_pred= model.predict(imf1_test)

In [ ]:
loss_1 = imf1_his.history['loss']
# 绘制训练曲线
plt.plot(loss_1, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse1 = np.sqrt(mean_squared_error(imf1_test, imf1_pred))
mae1 = mean_absolute_error(imf1_test, imf1_pred)

# 输出评估指标
print("RMSE:", rmse1)
print("MAE:", mae1)

#### IMF2

In [ ]:
imf2 = get_imf(train_imfs, 2)
imf2_test = get_imf(test_imfs, 2)

In [ ]:
imf2_his=model.fit(imf2, np.roll(imf2, -1), epochs=100, batch_size=128)

In [ ]:
imf2_pred= model.predict(imf2_test)

In [ ]:
loss_2 = imf2_his.history['loss']
# 绘制训练曲线
plt.plot(loss_2, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse2 = np.sqrt(mean_squared_error(imf2_test, imf2_pred))
mae2 = mean_absolute_error(imf2_test, imf2_pred)

# 输出评估指标
print("RMSE:", rmse2)
print("MAE:", mae2)

#### IMF3

In [ ]:
imf3 = get_imf(train_imfs, 3)
imf3_test = get_imf(test_imfs, 3)

In [ ]:
imf3_his=model.fit(imf3, np.roll(imf3, -1), epochs=100, batch_size=128)

In [ ]:
imf3_pred= model.predict(imf3_test)

In [ ]:
loss_3 = imf3_his.history['loss']
# 绘制训练曲线
plt.plot(loss_3, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse3 = np.sqrt(mean_squared_error(imf3_test, imf3_pred))
mae3 = mean_absolute_error(imf3_test, imf3_pred)

# 输出评估指标
print("RMSE:", rmse3)
print("MAE:", mae3)

#### IMF4

In [ ]:
imf4 = get_imf(train_imfs, 4)
imf4_test = get_imf(test_imfs, 4)

In [ ]:
imf4_his=model.fit(imf4, np.roll(imf4, -1), epochs=100, batch_size=128)

In [ ]:
imf4_pred= model.predict(imf4_test)

In [ ]:
loss_4 = imf4_his.history['loss']
# 绘制训练曲线
plt.plot(loss_4, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse4 = np.sqrt(mean_squared_error(imf4_test, imf4_pred))
mae4 = mean_absolute_error(imf4_test, imf4_pred)

# 输出评估指标
print("RMSE:", rmse4)
print("MAE:", mae4)

#### IMF5

In [ ]:
imf5 = get_imf(train_imfs, 5)
imf5_test = get_imf(test_imfs, 5)

In [ ]:
imf5_his=model.fit(imf5, np.roll(imf5, -1), epochs=100, batch_size=128)

In [ ]:
imf5_pred= model.predict(imf5_test)

In [ ]:
loss_5 = imf5_his.history['loss']
# 绘制训练曲线
plt.plot(loss_5, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse5 = np.sqrt(mean_squared_error(imf5_test, imf5_pred))
mae5 = mean_absolute_error(imf5_test, imf5_pred)

# 输出评估指标
print("RMSE:", rmse5)
print("MAE:", mae5)

#### IMF6

In [ ]:
imf6 = get_imf(train_imfs, 6)
imf6_test = get_imf(test_imfs, 6)

In [ ]:
imf6_his=model.fit(imf6, np.roll(imf6, -1), epochs=100, batch_size=128)

In [ ]:
imf6_pred= model.predict(imf6_test)

In [ ]:
loss_6 = imf6_his.history['loss']
# 绘制训练曲线
plt.plot(loss_6, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse6 = np.sqrt(mean_squared_error(imf6_test, imf6_pred))
mae6 = mean_absolute_error(imf6_test, imf6_pred)

# 输出评估指标
print("RMSE:", rmse6)
print("MAE:", mae6)

#### IMF7

In [ ]:
imf7 = get_imf(train_imfs, 7)
imf7_test = get_imf(test_imfs, 7)

In [ ]:
imf7_his=model.fit(imf7, np.roll(imf7, -1), epochs=100, batch_size=128)

In [ ]:
imf7_pred= model.predict(imf7_test)

In [ ]:
loss_7 = imf7_his.history['loss']
# 绘制训练曲线
plt.plot(loss_7, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse7 = np.sqrt(mean_squared_error(imf7_test, imf7_pred))
mae7 = mean_absolute_error(imf7_test, imf7_pred)

# 输出评估指标
print("RMSE:", rmse7)
print("MAE:", mae7)

#### Residue

In [ ]:
res_his=model.fit(train_residue, np.roll(train_residue, -1), epochs=100, batch_size=128)

In [ ]:
res_pred= model.predict(test_residue)

In [ ]:
loss_res = res_his.history['loss']
# 绘制训练曲线
plt.plot(loss_res, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse_res = np.sqrt(mean_squared_error(test_residue, res_pred))
mae_res = mean_absolute_error(test_residue, res_pred)

# 输出评估指标
print("RMSE:", rmse_res)
print("MAE:", mae_res)

## Prediction Result

In [ ]:
sum_pred = imf1_pred + imf2_pred + imf3_pred + imf4_pred + imf5_pred + imf6_pred + imf7_pred + res_pred
sum_test = imf1_test + imf2_test + imf3_test + imf4_test + imf5_test + imf6_test + imf7_test + test_residue.squeeze()

In [ ]:
sum_pred = pd.DataFrame({'pred': sum_pred.reshape(-1)})
sum_test = pd.DataFrame({'test': sum_test}).reset_index(drop=True)
comb_emd = pd.concat([sum_pred, sum_test], axis=1)

comb_emd.head()

In [ ]:
li_rv_comp = li['rv'][5885:].reset_index(drop=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 6))

# 绘制 Actual 曲线和 Predicted 曲线，共用同一个y轴
ax1.plot(li_rv_comp, label='Actual', alpha=0.5, color='blue')
ax1.plot(comb_emd['pred'], label='Predicted', color='red')
ax1.set_ylabel('波动率', fontsize = 15)

# 添加图例
fig.legend(loc="upper right",fontsize = 15)

plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel('时间',fontsize = 15)
plt.tight_layout()
plt.show()

# LSTM

In [ ]:
X = li.drop(columns=['rv'])
y = li['rv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 显示训练集和测试集的形状
print("训练集特征形状:", X_train.shape)
print("训练集目标形状:", y_train.shape)
print("测试集特征形状:", X_test.shape)
print("测试集目标形状:", y_test.shape)

In [ ]:
plt.figure(figsize=(15, 5))

# 绘制训练集目标变量曲线，设置 alpha 为 0.5
plt.plot(range(len(y_train)), y_train.values, label='Training Set', alpha=0.5, color='blue')

# 绘制测试集目标变量曲线
plt.plot(range(len(y_train), len(y_train) + len(y_test)), y_test.values, label='Test Set', color='blue')

# 添加图例、标题和标签
plt.legend()
plt.title('测试集划分情况')
plt.xlabel('时间')
plt.ylabel('Target Variable')

plt.show()

## Model Training

In [ ]:
li_his = model.fit(X_train, y_train,epochs=100, batch_size=64)

In [ ]:
rv_pred = model.predict(X_test)

In [ ]:
loss_rv = li_his.history['loss']
# 绘制训练曲线
plt.plot(loss_rv, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
rmse_rv = np.sqrt(mean_squared_error(y_test, rv_pred))
mae_rv = mean_absolute_error(y_test, rv_pred)

# 输出评估指标
print("RMSE:", rmse_rv)
print("MAE:", mae_rv)

## Prediction Result

In [ ]:
rv_pred = pd.DataFrame({'pred': rv_pred.reshape(-1)})
y_test = pd.DataFrame({'test': y_test}).reset_index(drop=True)
comb = pd.concat([rv_pred, y_test], axis=1)

comb.head()

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(comb['pred'], label='Predicted', color='blue')
plt.plot(comb['test'], label='Actual', color='red',alpha=0.5)

# 添加图例、标题和标签
plt.legend()
plt.title('Actual vs Predicted')
plt.xlabel('Index')
plt.ylabel('Value')

plt.show()

# Value-at-Risk

## GARCH

首先，对收益率进行平稳性以及异方差检验

### ADF

In [ ]:
ADF(li['return'])

### Engle

In [ ]:
lishift=li['return'].shift(1).fillna(np.mean(li['return']))

In [ ]:
eg_result = engle_granger(li['return'], lishift)

print('\nEngle-Granger Statistic: {:.4f}'.format(eg_result.stat))
print('p-value: {:.4f}'.format(eg_result.pvalue))
print('Null Hypothesis: {}'.format(eg_result.null_hypothesis))
print('Alternative Hypothesis: {}'.format(eg_result.alternative_hypothesis))

### Spliting dataset

In [ ]:
train_re, test_re = train_test_split(li['return'], test_size=0.2, shuffle=False)

In [ ]:
print(train_re.shape)
print(test_re.shape)

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(np.arange(len(train_re)), train_re, color='blue', label='训练目标')

plt.plot(np.arange(len(train_re), len(train_re) + len(test_re)),
         test_re, color='black', label='测试目标')

plt.title('收益率 数据划分情况', fontsize=20)
plt.xlabel('时间', fontsize=15)
plt.legend(loc='best')

locator = MultipleLocator(1000)
plt.gca().xaxis.set_major_locator(locator)
plt.show()

### Training model

In [ ]:
def fit_vol_model(y, model_params, train_test_split = None):
    model = arch_model(y, **model_params)
    
    fitted = model.fit(disp = "off")
    train_y = y
        
    train_pred = fitted.conditional_volatility.dropna()
    train_resid = fitted.resid / train_pred
    
    return {
        "Test y": train_y,
        "Pred": train_pred,
        "Resid": train_resid,
        "Fitted": fitted,
        "Model": model
    }

garch_params = {"p": 1, "q": 1, "vol": "Garch", "dist": "skewt"}

In [ ]:
vol_results = fit_vol_model(test_re*100, garch_params)

### Vol Comparison with LSTM and EMD-LSTM

In [ ]:
vol_test= pd.DataFrame(vol_results['Test y'])
vol_pred = pd.DataFrame(vol_results['Pred'])
vol_resid= pd.DataFrame(vol_results['Resid'])
res = vol_results['Fitted']
res

In [ ]:
vol_pred_comp = vol_pred.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(15, 6))

# 绘制 Actual 曲线
plt.plot(li_rv_comp, label='Actual', color='blue')

# 绘制 GARCH 曲线
plt.plot(vol_pred_comp, label='GARCH', color='green')

# 绘制 LSTM 预测曲线
plt.plot(comb['pred'], label='LSTM', color='purple')

# 绘制 EMD-LSTM 曲线
plt.plot(comb_emd['pred'], label='EMD-LSTM', color='red')

# 添加图例、标题和标签，并调整图例的字体大小
plt.xlabel('时间')
plt.ylabel('已实现波动率')
plt.legend(prop={'size': 12})

# 显示图形
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 6))

# 绘制 vol_pred 列的数据
ax1.plot(vol_pred, label='GARCH', color='blue')
ax1.plot(li['rv'][5885:], label='实际波动率', color='red', alpha=0.75)

# 设置标签和标题
ax1.set_xlabel('时间', fontsize=14)
ax1.set_ylabel('波动率值', fontsize=14)
ax1.set_title('GARCH模型波动率预测效果图', fontsize=16)

# 添加图例
fig.legend(loc="upper right", fontsize='large')

plt.tight_layout()
plt.show()

### VaR Prediction

In [ ]:
garch_params

In [ ]:
res = vol_results['Fitted']

In [ ]:
am = arch_model(li['return']*1000, p = 1, q = 1, o = 1, vol = 'GARCH', dist = 't')
res = am.fit(disp='off')

In [ ]:
forecasts = res.forecast(start=5885)
cond_mean = forecasts.mean[:]
cond_var = forecasts.variance[:]
q = am.distribution.ppf([0.01, 0.05], res.params[5])
print(q)

In [ ]:
print('cond_mean shape:', cond_mean.shape)
print('cond_var shape:', cond_var.shape)
print('q shape:', q.shape)

In [ ]:
# 计算VaR
value_at_risk = -cond_mean.values - np.sqrt(cond_var).values * q[None, :]
value_at_risk = pd.DataFrame(value_at_risk, columns=["1%", "5%"], index=cond_var.index)

In [ ]:
value_at_risk

In [ ]:
merged_df = pd.concat([li['return'][5885:]*1000, value_at_risk], axis=1)
merged_df.reset_index(drop=True, inplace=True)
merged_df

In [ ]:
plt.figure(figsize=(15, 6))

# 绘制 LI Return 列
plt.plot(merged_df.index, merged_df['return'], label='LI Return', color='blue',alpha = 0.5)

# 绘制 Value at Risk 列
plt.plot(merged_df.index, -merged_df['1%'], label='VaR 99%', color='red')
plt.plot(merged_df.index, -merged_df['5%'], label='VaR 95%', color='green')

# 添加图例、标题和标签
plt.legend(fontsize=18)
plt.xlabel('时间',fontsize = 18)
plt.ylabel('Value',fontsize = 18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
ax = value_at_risk.plot(legend=False, figsize=(15,6))
xl = ax.set_xlim(value_at_risk.index[0], value_at_risk.index[-1])

rets = li['return'][5885:]*1000
rets.name = 'LI Return'

c = []
for idx in value_at_risk.index:
    if rets[idx] > -value_at_risk.loc[idx, '5%']:
        c.append('#000000')
    elif rets[idx] < -value_at_risk.loc[idx, '1%']:
        c.append('#BB0000')
    else:
        c.append('#BB00BB')
        
c = np.array(c, dtype='object')

labels = {
    
    '#BB0000': '1% Exceedence',
    '#BB00BB': '5% Exceedence',
    '#000000': 'No Exceedence'
}

markers = {'#BB0000': 'x', '#BB00BB': 's', '#000000': 'o'}

for color in np.unique(c):
    sel = c == color
    ax.scatter(
        rets.index[sel],
        -rets.loc[sel],
        marker=markers[color],
        c=c[sel],
        label=labels[color])
    
ax.legend(frameon=False, ncol=3)
plt.xlabel('时间',fontsize = 18)
plt.ylabel('Value',fontsize = 18)
plt.show()

## EMD-LSTM

OMG finally we're here! Feels like centries!

To calculate VaR, we need:

\begin{equation}
VaR_{t} = \mu_{t} + \sigma_{t} Z_{\alpha}
\end{equation}

where, Where $Z_{\alpha}$represents the critical value, $\mu_{t}$ denotes the average return of the underlying asset, and $\sigma_{t}$ is the estimated volatility. 

In [ ]:
mu = cond_mean
sigma =pd.DataFrame(comb_emd['pred'])

print('mu:',type(mu))
print('sigma:',type(sigma))

In [ ]:
var_dl = -mu.values - sigma.values * q[None, :]
var_dl = pd.DataFrame(var_dl, columns=["1%", "5%"], index=sigma.index)

In [ ]:
lire = li['return'][5885:]
lire.reset_index(drop = True, inplace =True)

In [ ]:
dl = pd.concat([lire, var_dl], axis=1)
dl.reset_index(drop=True, inplace=True)
dl

### VaR Prediction

In [ ]:
plt.figure(figsize=(15, 6))

# 绘制 LI Return 列
plt.plot(dl.index, dl['return']*10, label='LI Return', color='blue',alpha = 0.5)

# 绘制 Value at Risk 列
plt.plot(dl.index,-dl['1%'], label='99% VaR', color='red')
plt.plot(dl.index,-dl['5%'], label='95% VaR', color='green')

# 添加图例、标题和标签
plt.legend(fontsize=18)
plt.xlabel('时间',fontsize = 18)
plt.ylabel('Value',fontsize = 18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
ax = var_dl.plot(legend=False, figsize=(15,6))
xl = ax.set_xlim(var_dl.index[0], var_dl.index[-1])

rets = li['return'][5885:]*1000
rets.name = 'LI Return'

c = []
for idx in value_at_risk.index:
    if rets[idx] > -value_at_risk.loc[idx, '5%']:
        c.append('#000000')
    elif rets[idx] < -value_at_risk.loc[idx, '1%']:
        c.append('#BB0000')
    else:
        c.append('#BB00BB')
        
c = np.array(c, dtype='object')

labels = {
    
    '#BB0000': '1% Exceedence',
    '#BB00BB': '5% Exceedence',
    '#000000': 'No Exceedence'
}

markers = {'#BB0000': 'x', '#BB00BB': 's', '#000000': 'o'}

for color in np.unique(c):
    sel = c == color
    ax.scatter(
        rets.index[sel],
        -rets.loc[sel],
        marker=markers[color],
        c=c[sel],
        label=labels[color])
    
ax.legend(frameon=False, ncol=3)
plt.xlabel('时间',fontsize = 18)
plt.ylabel('Value',fontsize = 18)
plt.show()

# Kupiec Test

In [ ]:
def kupiec_test(actual_losses, var_predictions, confidence_level):
    # 计算失败次数
    failures = (actual_losses > var_predictions).sum()
    # 总观察次数
    total_obs = len(actual_losses)
    # 预期失败次数
    expected_failures = total_obs * (1 - confidence_level)
    # 计算似然比统计量
    lr_stat = -2 * np.log(binom.pmf(failures, total_obs, 1 - confidence_level)) + 2 * np.log(binom.pmf(failures, total_obs, 1 - failures / total_obs))
    # 计算卡方分布的临界值
    critical_value = chi2.ppf(1 - confidence_level, 1)

    return lr_stat,critical_value,failures

In [ ]:
actual_losses = li['return'][5885:].reset_index(drop=True)
actual_losses

## 95%

### EMD-LSTM

In [ ]:
var_predictions = np.squeeze(sum_pred)
confidence_level = 0.95
# 进行 Kupiec 检验
lr_stat, critical_value,failures = kupiec_test(actual_losses, var_predictions, confidence_level)

print("Likelihood Ratio Statistic:", lr_stat)
print("Critical Value:", critical_value)
print("failures", failures)
print("Reject null hypothesis?" , "Yes" if lr_stat > critical_value else "No")

### GARCH

In [ ]:
var_predictions = merged_df['5%']
confidence_level = 0.95

# 进行 Kupiec 检验
lr_stat, critical_value,failures = kupiec_test(actual_losses, var_predictions, confidence_level)

# 输出结果
print("Likelihood Ratio Statistic:", lr_stat)
print("Critical Value:", critical_value)
print("failures", failures)
print("Reject null hypothesis?" , "Yes" if lr_stat > critical_value else "No")

## 99%

### EMD-LSTM

In [ ]:
var_predictions = np.squeeze(sum_pred)
confidence_level = 0.99

# 进行 Kupiec 检验
lr_stat, critical_value,failures = kupiec_test(actual_losses, var_predictions, confidence_level)

# 输出结果
print("Likelihood Ratio Statistic:", lr_stat)
print("Critical Value:", critical_value)
print("failures", failures)
print("Reject null hypothesis?" , "Yes" if lr_stat > critical_value else "No")

### GARCH

In [ ]:
var_predictions = merged_df['1%']
confidence_level = 0.99

# 进行 Kupiec 检验
lr_stat, critical_value, failures = kupiec_test(actual_losses, var_predictions, confidence_level)

# 输出结果
print("Likelihood Ratio Statistic:", lr_stat)
print("Critical Value:", critical_value)
print("failures", failures)
print("Reject null hypothesis?" , "Yes" if lr_stat > critical_value else "No")

# 致谢

这篇论文是金融学主修学位的毕业论文。

老实说，首先要感谢自己辅修了应用统计学的学位，不然毕业论文选题不知道要怎么办才能叫本科生创新。当然这也给了我一个启发：如果你想要在现有的专业领域里有新的路子可以走，学多点，学杂点，总归是好的。

比如在我的同系的同学们最多用Python简单数据分析的时候，我可以直接手搓一个深度学习模型来完成我的毕业论文，虽然对于真正这个领域的同学来说简直是班门弄斧。

但至少我有机会看见新的世界，以及新的可能性 :)

总之，谢谢的想法已经在另一篇里道尽了，写完这一篇反而有种如释重负的感觉。

但我还是很感谢一直以来陪伴我走到这里的人。

因为这些朋友的存在，让我学会了如何去好好爱身边的人，好好去表达以及接受这些爱意，珍视每一份难得的心意。

虽然我依旧是一个糟糕的人，但谢谢你们，因为你们的存在让我感觉我没有糟糕到无药可救。 :3

<div style="text-align: right;">
歪，2024年4月。
</di月留